# Here you need to add the paper title, and then a small block of general explanation for a binder

# Experiments on neuromuscular coupling of the diaphragm

Import libraries

In [ ]:
from scipy import interpolate as interp
import scipy
import pandas as pd
import glob
import os
import sys
import numpy as np
import numpy.matlib
import matplotlib.pyplot as plt
import ipywidgets as widgets
from datetime import datetime
%matplotlib widget

# The cell below will need to be changed for a generalized binder

In [ ]:
# import helper functions
sys.path.insert(0, '..')

import resurfemg.preprocessing.ecg_removal as ecg_rm
import resurfemg.preprocessing.envelope as evl
import resurfemg.preprocessing.filtering as filt
import resurfemg.postprocessing.features as feat

from resurfemg.config.config import Config
config = Config()

# get new changes in our library i.e. the tmsisdk
sys.path.insert(0, '..')

from resurfemg.data_connector.tmsisdk_lite import Poly5Reader

# Initiation of output folder for experiments

In [ ]:
# Output data - General path to direcotroy for saving .csvs and plots
main_output_dir = config.get_directory('preprocessed') + \
                    '/output_directory_name'
if not os.path.exists(main_output_dir):
    os.makedirs(main_output_dir)

# Root directory for patient data
root_patient_data_directory = \
    config.get_directory('root_patient_data_directory')

patient_idx = 0

# Patient data selection

In [ ]:
# Reruns should be done from this cell as the start

# Expected data structure:
# - Patient_01
# -- Measurement_date_XXXX_XX_01
# --- 001_Baseline
# --- 002_PEEP_step_01
# --- 003_PEEP_step_02
# --- 004_PEEP_step_03
# --- 005_PEEP_step_04
# -- Measurement_date_XXXX_XX_03
# --- 001_Baseline
# --- 002_PEEP_step_01
# --- 003_PEEP_step_02
# --- 004_PEEP_step_03
# --- 005_PEEP_step_04
# -- Patient_02
# -- Measurement_date_XXXX_XX_01
# etc.

In [ ]:
# Select the folder of the patient of interest
# Run this cell once per patient

patient_folders = \
    glob.glob(root_patient_data_directory + '/**/', recursive=False)
patients = []

for folder in patient_folders:
    name = folder.split("\\")[-2]
    patients.append(name)

btn_pt = widgets.Dropdown(  
    options=patients,
    value=patients[patient_idx],
    description='Select patient:',
    disabled=False,
)

date_idx = 0

display(btn_pt)

In [ ]:
# Select the folder of the PEEP trial of interest of the selected patient
# measurement_date ~ PEEP-trial
# Run this cell once per patient/PEEP trial

patient = btn_pt.value
patient_idx =btn_pt.index

measurement_folders = glob.glob(root_patient_data_directory 
                                + '/' + patient 
                                + '/**/',
                                recursive=False
                                )
measurement_dates = []

for folder in measurement_folders:
    name = folder.split("\\")[-2]
    measurement_dates.append(name)

btn_measurement = widgets.Dropdown(
    options=measurement_dates,
    value=measurement_dates[date_idx],
    description='Select measurement date:',
    disabled=False,
)
display(btn_measurement)

In [ ]:
# Identify all recordings available for the selected patient/measurement_date
# Rerun this ceel for each new PEEP-trial, as it also empties output parameter 
# list (big_data_list)!

# Identify all PEEP-step folders:
measurement_date = btn_measurement.value
date_idx = btn_measurement.index

windows_str = (root_patient_data_directory + '/' + patient + '/' 
               + measurement_date)
root_emg_directory = os.path.join(windows_str)

emg_pattern = os.path.join(root_emg_directory, '**/*.Poly5')
emg_and_vent_files = glob.glob(emg_pattern, recursive=True)

emg_files = []
vent_files = []

for file in emg_and_vent_files:
    if 'Draeger' in file:
        vent_files.append(file)
    else:
        emg_files.append(file)
list_of_numbers_strung = []

for i in range(len(emg_files)):
    list_of_numbers_strung.append(emg_files[i].split('\\')[-2])

# Initialise the analysis: empty the output parameter list and start at the 
# first recorded PEEP step (index -4)
big_data_list = []
PEEP_step_idx = -4

Now you can pick a file from the list, which have been numbered.

In [ ]:
# Select the PEEP step of interest. The selection menu initialises at the third 
# but last (index -4) recording

btn_PEEP_step = widgets.Dropdown(
    options=list_of_numbers_strung,
    value=str(emg_files[PEEP_step_idx].split('\\')[-2]),
    description='Picked File:',
    disabled=False,
)
display(btn_PEEP_step)

In [ ]:
# Process the PEEP step selection 
PEEP_step_chosen = btn_PEEP_step.value
PEEP_step_idx = int(btn_PEEP_step.index)
emg_file_chosen = emg_files[PEEP_step_idx]
vent_file_chosen = vent_files[PEEP_step_idx]
print("The files you chose are:\n", emg_file_chosen, '\n', vent_file_chosen)

In [ ]:
# Create a directory per PEEP trial to save the outputs .CSVs and plots to.
# Outputs are structured per patient/measurement_date:
# - Patient_01
# -- Measurement_date_XXXX_XX_01
# -- Measurement_date_XXXX_XX_02
# - Patient_02
# etc.

patient = emg_files[PEEP_step_idx].split('\\')[-3].split('/')[-2]

if not os.path.exists(main_output_dir + '/' + patient + '/'):
    os.makedirs(main_output_dir + '/' + patient + '/')

if not os.path.exists(main_output_dir + '/' + patient 
                      + '/' + measurement_date + '/'
):
    os.makedirs(main_output_dir + '/' + patient + '/' 
                + measurement_date + '/')

In [ ]:
# Load the EMG and ventilator data recordings from the selected folders.
data_emg = Poly5Reader(emg_file_chosen)
data_vent = Poly5Reader(vent_file_chosen)
data_emg_samples = data_emg.samples[:data_emg.num_samples]
emg_sample_rate = data_emg.sample_rate
data_vent_samples = data_vent.samples[:data_vent.num_samples]
vent_sample_rate = data_vent.sample_rate

# Define the time series of the EMG and ventilator recordings
y_emg = data_emg_samples
y_vent = data_vent_samples

# Define the the axes
t_emg = [i/emg_sample_rate for i in range(len(y_emg[0, :]))]
t_vent = [i/vent_sample_rate for i in range(len(y_vent[0, :]))]

In [ ]:
# Window including all end-expiratory occlusion manoeuvres (Pocc)
t_start_default = t_vent[-1]-100
t_end_default = t_vent[-1]-10

# Gating settings
gate_width_default = 0.10
gate_threshold_default = 0.30
gate_ECG_shift_default = -10
gate_twice = False

# Peak detection settings
time_shift_default = 0.5    # Delay between sEMG- and Pocc peak
P_occ_prominence_factor_default = 0.8
EMG_di_prominence_factor_default = 0.5
EMG_para_prominence_factor_default = 0.5

In [ ]:
# Plot the raw data
fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(10, 6))

axis[0, 0].grid(True)
axis[0, 0].plot(t_emg, y_emg[0])
axis[0, 0].set(title='sEMG leads')
axis[0, 0].set_ylabel('ECG (uV)')
axis[1, 0].plot(t_emg, y_emg[1])
axis[1, 0].set_ylabel('sEMGdi (uV)')
axis[1, 0].set_xlabel('t (s)')

axis[0, 1].set(title='Ventilator data')
axis[0, 1].grid(True)
axis[0, 1].plot(t_vent, y_vent[0])
axis[0, 1].set_ylabel('Paw (cmH2O)')
axis[1, 1].plot(t_vent, y_vent[1])
axis[1, 1].set_ylabel('F (L/min)')
axis[2, 1].plot(t_vent, y_vent[2])
axis[2, 1].set_ylabel('V (mL)')
axis[2, 1].set_xlabel('t (s)')

Here pick the part of the signal you want to look at. It is advised to never start at zero, or end after the signal, and the widgets have been constrained to do this.

In [ ]:
# Enter the start and end time of the window in which the occlusions occur. Make
#  sure to take some margin around the peaks in the order of a couple of seconds
# to allow for the filtering algorithms to work properly. On the other hand, if 
# the window is to broad, the algorithms will also take longer to evaluate.

start_widget = widgets.BoundedFloatText(
    value=np.floor(t_start_default),
    min=1,
    max=t_vent[-1],
    step=1,
    description='Start:',
    disabled=False
)
end_widget = widgets.BoundedFloatText(
    value=np.floor(t_end_default),
    min=1,
    max=t_vent[-1],
    step=1,
    description='End:',
    disabled=False
)
widgets.VBox([start_widget, end_widget])

In [ ]:
# Examine at the selected window:
# 1. Does it contain all occlusions? If not, adapt the start and/or end time to 
#    include all Pocc manoeuvres.
# 2. Does the window contain limited data before and after the occlusions 
#    (<10 s). If noy, you can adapt the start and/or end time to optimise 
#    calculation speed.

start = start_widget.value  
end = end_widget.value  
if ((0 > start) | (t_vent[-1] < start) | (0 > end) | (t_vent[-1] < end) 
    | (start >= end)):    
    print('Please make sure that 1) the start and end times are within the'  
          +' limits of the recording and 2) that the start time is before the' 
          + ' end time.')
else:
    # Plot the selected window.
    start_s = int(float(start)* emg_sample_rate)
    end_s = min([int(float(end)*emg_sample_rate), len(y_emg[0,:])-1])
    start_vent_s = int(float(start)* vent_sample_rate)
    end_vent_s = min(
        [int(float(end)* vent_sample_rate), len(y_vent[0,:])-1]
    )

    fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(12, 6))
    axis[0, 0].set(title='sEMG leads')
    axis[0, 0].grid(True)
    axis[0, 0].plot(t_emg[int(start_s):int(end_s)], 
                    y_emg[0][int(start_s):int(end_s)])
    axis[0, 0].set_ylabel('ECG (uV)')
    axis[1, 0].plot(t_emg[int(start_s):int(end_s)], 
                    y_emg[1][int(start_s):int(end_s)])
    axis[1, 0].set_ylabel('sEMGdi (uV)')
    axis[1, 0].set_xlabel('t (s)')
    axis[0, 1].set(title='Ventilator data')
    axis[0, 1].grid(True)
    axis[0, 1].plot(
        t_vent[int(start_vent_s):int(end_vent_s)], 
        y_vent[0][int(start_vent_s):int(end_vent_s)]
    )
    axis[0, 1].set_ylabel('Paw (cmH2O)')
    axis[1, 1].plot(
        t_vent[int(start_vent_s):int(end_vent_s)], 
        y_vent[1][int(start_vent_s):int(end_vent_s)]
    )
    axis[1, 1].set_ylabel('F (L/min)')
    axis[2, 1].plot(
        t_vent[int(start_vent_s):int(end_vent_s)], 
        y_vent[2][int(start_vent_s):int(end_vent_s)]
    )
    axis[2, 1].set_ylabel('V (mL)')
    axis[2, 1].set_xlabel('t (s)')

# Pre-processing

In [ ]:
# Eliminate the baseline wander from the data using a band-pass filter
bd_filtered_file_data = filt.emg_bandpass_butter_sample(
    data_emg_samples, 1, 500, 2048, output='sos')
    
processed_data_emg_di = bd_filtered_file_data[1]

In [ ]:
# Gating settings

# Window size to plot the gating results from
plot_window = 5         

# QRS peak detection settings
# peak_fraction = 0.30 (default value)
peak_fraction = gate_threshold_default     # Minimal relative peak height
peak_width = 0.001
peak_dist = int(emg_sample_rate/3)

# Shift of gated in samples relative to detected peaks      
# ECG_shift = -10 (default value)
ECG_shift = gate_ECG_shift_default

# Gate width in seconds
# gate_width = 0.10 (default value)
gate_width = gate_width_default

In [ ]:
# Apply QRS gating

# Detect ECG peaks on 1Hz high-pass filtered signals
ecg = bd_filtered_file_data[0]
ecg_rms = evl.full_rolling_rms(ecg, 10)
max_ecg_rms = max(
    ecg_rms[int(start_s):int(start_s+plot_window*emg_sample_rate)])
min_ecg_rms = min(
    ecg_rms[int(start_s):int(start_s+plot_window*emg_sample_rate)])
peak_height = peak_fraction*(max_ecg_rms - min_ecg_rms)

ECG_peaks, properties  = scipy.signal.find_peaks(
    ecg_rms, 
    height=peak_height, 
    width=peak_width*emg_sample_rate, 
    distance=peak_dist
)
ECG_peaks = ECG_peaks - ECG_shift

# Filter EXG signals 20Hz high pass
ECG_h20 = filt.emg_bandpass_butter_sample(ecg, 20, 500, 2048, output='sos')
emg_di_h20 = filt.emg_bandpass_butter_sample(
    processed_data_emg_di, 20, 500, 2048, output='sos')

# Gate ECG and EMG signal
# Fill methods 0: Zeros, 1: Interpolate start-end, 2: Average prior data, 
# 3: Moving average
gate_fill_method = 3    
gate_width_samples = int(gate_width*emg_sample_rate)

gate_samples = list()
for i in range(len(ECG_peaks)):
    for k in range(
        int(ECG_peaks[i]-gate_width_samples/2),
        int(ECG_peaks[i]+gate_width_samples/2)
    ):
        gate_samples.append(k)

ECG_gated = ecg_rm.gating(
    ECG_h20, ECG_peaks, gate_width=gate_width_samples, method=3)
emg_di_gated = ecg_rm.gating(
    emg_di_h20, ECG_peaks, gate_width=gate_width_samples, method=3)


In [ ]:
# Apply gating twice, e.g., in case of splitted QRS complex or pacing
if gate_twice == True:
    # Detect ECG peaks on 1Hz high-pass filtered signals
    ecg = ECG_gated
    ecg_rms = evl.full_rolling_rms(ecg, 10)
    max_ecg_rms = max(
        ecg_rms[int(start_s):int(start_s+plot_window*emg_sample_rate)])
    min_ecg_rms = min(
        ecg_rms[int(start_s):int(start_s+plot_window*emg_sample_rate)])
    peak_height = peak_fraction*(max_ecg_rms - min_ecg_rms)

    ECG_peaks, properties  = scipy.signal.find_peaks(
        ecg_rms, 
        height=peak_height, 
        width=peak_width*emg_sample_rate, 
        distance=peak_dist
    )
    ECG_peaks = ECG_peaks - ECG_shift

    # Gate ECG and EMG signal
    # Fill methods: 0: Zeros, 1: Interpolate start-end, 2: Average prior data, 
    # 3: Moving average
    gate_fill_method = 3    
    gate_width_samples = int(gate_width*emg_sample_rate)

    gate_samples = list()
    for i in range(len(ECG_peaks)):
        for k in range(
            int(ECG_peaks[i]-gate_width_samples/2),
            int(ECG_peaks[i]+gate_width_samples/2)
        ):
            gate_samples.append(k)

    ECG_gated = ecg_rm.gating(
        ECG_gated, ECG_peaks, gate_width=gate_width_samples, method=3)
    emg_di_gated = ecg_rm.gating(
        emg_di_gated, ECG_peaks, gate_width=gate_width_samples, method=3)

In [ ]:
# Plot gating result

fig, axis = plt.subplots(nrows=4, ncols=1, figsize=(12, 6))
start_i = int(start_s)
end_i = int(start_s+plot_window*2048)
axis[0].grid(True)
axis[0].plot(t_emg[start_i:end_i], bd_filtered_file_data[0, start_i:end_i])
axis[0].set_ylabel('raw ECG (uV)')
axis[0].set(title='leads in EMG')
axis[1].grid(True)
axis[1].plot(t_emg[start_i:end_i], ECG_gated[start_i:end_i])
axis[1].set_ylabel('ECG gated (uV)')
axis[2].grid(True)
axis[2].plot(t_emg[start_i:end_i], emg_di_gated[start_i:end_i])
axis[2].set_ylabel('EMGdi gated (uV)')
axis[3].grid(True)
axis[3].plot(t_emg[start_i:end_i], ecg_rms[start_i:end_i])
axis[3].set_ylabel('ECG rms (uV)')
axis[3].hlines(
    y=peak_height, 
    xmin=t_emg[start_i], 
    xmax=t_emg[end_i],
    color = "C1"
)

for idx in range(len(ECG_peaks)):
    if ((ECG_peaks[idx] > int(start_s)) 
        & (ECG_peaks[idx] < int(start_s+plot_window*2048))):
        axis[3].plot(t_emg[ECG_peaks[idx]], ecg_rms[ECG_peaks[idx]],'rx')


Calculate root mean squared (RMS)

In [ ]:
# Calculate the moving-RMS over the sEMG signal
RMS_window_ms = 200
RMS_windows_samp = int(RMS_window_ms / 1000 *  emg_sample_rate)

RMS_data_emg_di = evl.full_rolling_rms(emg_di_gated, RMS_windows_samp)

# Plot RMS results
fig, axis = plt.subplots(nrows=3, ncols=1, figsize=(12, 6))
axis[0].set(title='leads in EMG')
axis[0].set_ylabel('sEMGdi (uV)')
axis[0].grid(True)
axis[0].plot(t_emg[int(start_s):int(end_s)], 
             processed_data_emg_di[int(start_s):int(end_s)])
axis[0].plot(t_emg[int(start_s):int(end_s)], 
             RMS_data_emg_di[int(start_s):int(end_s)])

axis[1].set_xlabel('t (s)')
axis[1].set_ylabel('RMS{sEMGdi} (uV)')
axis[1].plot(t_emg[int(start_s):int(end_s)], 
             RMS_data_emg_di[int(start_s):int(end_s)])
axis[2].set_ylabel('Paw (cmH2O)')
axis[2].grid(True)
axis[2].plot(t_vent[int(start_vent_s):int(end_vent_s)], 
             y_vent[0][int(start_vent_s):int(end_vent_s)])

# Feature extraction

In [ ]:
# Detect the set PEEP level

# Find end-expiration samples by finding the minimal values in V_vent
V_vent_up_to_start = y_vent[2][:int(start_vent_s)]
V_ee_PKS, _ = scipy.signal.find_peaks(-V_vent_up_to_start)

# Calculate PEEP as the median value of Paw at end-expiration
PEEP_set = (np.median(y_vent[0, V_ee_PKS])-1)//2*2+1

PEEP_set

In [ ]:
# Old moving baseline on pressure and EMG signals for baseline crossing 
# detection in accordance with Graßhoff et al. (2021)

baseline_W_emg = 5 * emg_sample_rate  # window length
baseline_W_vent = 5 * vent_sample_rate  # window length

emg_di_rolling_base_line_old = np.zeros(
    (len(RMS_data_emg_di[int(start_s):int(end_s)]), ))

for idx in range(0, int(end_s)-int(start_s), int(emg_sample_rate/5)):
    start_i = max([int(start_s), int(start_s)+idx-int(baseline_W_emg/2)])
    end_i = min([int(end_s), int(start_s)+idx+int(baseline_W_emg/2)])
    baseline_value_emg_di = np.percentile(
        RMS_data_emg_di[start_i:end_i], 33)
    
    for i in range(idx, 
                   min([idx+int(emg_sample_rate/5), int(end_s)-int(start_s)])
    ):
        emg_di_rolling_base_line_old[i] = baseline_value_emg_di

In [ ]:
# Augmented moving baseline for EMG signals for baseline crossing detection

# Baseline windows
baseline_W_emg = int(7.5 * emg_sample_rate)  # window length
baseline_W_vent = int(7.5 * vent_sample_rate)  # window length


# Calculate the default moving baselines
P_rolling_base_line = np.zeros(
    (len(y_vent[0, int(start_vent_s):int(end_vent_s)]), ))
emg_di_rolling_base_line = np.zeros(
    (len(RMS_data_emg_di[int(start_s):int(end_s)]), ))

# Calculate the moving baseline over the Paw signal
for idx in range(0, 
                 int(end_vent_s)-int(start_vent_s), 
                 int(vent_sample_rate/5)
):
    start_i = max([int(start_vent_s), int(
        start_vent_s)+idx-int(baseline_W_vent/2)])
    end_i = min([int(end_vent_s), int(start_vent_s) +
                idx+int(baseline_W_vent/2)])
    P_rolling_base_line[idx] = np.percentile(y_vent[0, start_i:end_i], 33)

    baseline_value_P = np.percentile(y_vent[0, start_i:end_i], 33)
    for i in range(idx, 
            min([idx+int(vent_sample_rate/5), 
                 int(end_vent_s)-int(start_vent_s)])
    ):
        P_rolling_base_line[i] = baseline_value_P

# Calculate the "default" moving baseline over the sEMG data over a 7.5s window
for idx in range(0, int(end_s)-int(start_s), int(emg_sample_rate/5)):
    start_i = max([int(start_s), int(start_s)+idx-int(baseline_W_emg/2)])
    end_i = min([int(end_s), int(start_s)+idx+int(baseline_W_emg/2)])
    baseline_value_emg_di = np.percentile(
        RMS_data_emg_di[start_i:end_i], 33)
    
    for i in range(idx, 
                   min([idx+int(emg_sample_rate/5), int(end_s)-int(start_s)])
    ):
        emg_di_rolling_base_line[i] = baseline_value_emg_di

# Calculate the augmented moving baselines for the sEMG data
di_baseline_series = pd.Series(emg_di_rolling_base_line)
di_baseline_std = di_baseline_series.rolling(baseline_W_emg, 
                                 min_periods=1, 
                                 center=True).std().values
di_baseline_mean = di_baseline_series.rolling(baseline_W_emg, 
                                 min_periods=1, 
                                 center=True).mean().values

# Augmented signal := EMG + abs(dEMG/dt_smoothed)
ma_window = window=emg_sample_rate//2
augmented_perc = 25
perc_window = emg_sample_rate

y_di_RMS = RMS_data_emg_di[int(start_s):int(end_s)]
s_di = pd.Series(y_di_RMS - emg_di_rolling_base_line)
EMG_di_MA = s_di.rolling(window=ma_window, center=True).mean().values
dEMG_di_dt = (EMG_di_MA[1:] - EMG_di_MA[:-1] ) * emg_sample_rate
EMG_di_aug = y_di_RMS[:-1] + np.abs(dEMG_di_dt)

# Rolling baseline of augmented signals
emg_di_aug_rolling_base_line = np.zeros(
    (len(RMS_data_emg_di[int(start_s):int(end_s)-1]), ))

# Run the moving median filter over the augmented signal to obtain the baseline
for idx in range(0, int(end_s-1)-int(start_s), perc_window):
    start_i = max([0, idx-int(baseline_W_emg)])
    end_i = min([int(end_s-start_s-1), idx+int(baseline_W_emg)])

    baseline_value_emg_di = np.nanpercentile(
        EMG_di_aug[start_i:end_i], augmented_perc)
    
    for i in range(idx, 
                   min([idx+int(perc_window), int(end_s-1)-int(start_s)])
    ):
        emg_di_aug_rolling_base_line[i] = 1.2 * baseline_value_emg_di

Detect the end-expiratory occlusions (Pocc)

In [ ]:
# # Peak detection settings
# P_occ_prominence_factor = 0.8 (default value)
P_occ_prominence_factor = P_occ_prominence_factor_default

P_occ_prominence_factor

In [ ]:
# Detect (negative) occlussion peaks in pressure signal
treshold = 0
width = int(0.1 * vent_sample_rate)
prominence = P_occ_prominence_factor *np.abs( PEEP_set - min(y_vent[0]))
height = -(PEEP_set - P_occ_prominence_factor * 
           np.abs( PEEP_set - min(y_vent[0])))
distance = int(0.5 * vent_sample_rate)

x = y_vent[0, int(start_vent_s):int(end_vent_s)]
P_occ_peaks, properties  = scipy.signal.find_peaks(
    -x, 
    height=height, 
    prominence=prominence, 
    width=width, 
    distance=distance
)

# Detect Pocc on- and offset using moving baseline crossings
PEEP_crossings_idx = np.argwhere(np.diff(np.sign(x - P_rolling_base_line)) != 0)

P_occ_starts = np.zeros((len(P_occ_peaks),), dtype=int)
P_occ_ends = np.zeros((len(P_occ_peaks),), dtype=int)
for idx in range(len(P_occ_peaks)):
    PEEP_crossings_idx_sub = np.sign(
        x[:P_occ_peaks[idx]] - P_rolling_base_line[:P_occ_peaks[idx]])
    a = np.argmin(
        P_occ_peaks[idx] - np.argwhere(np.diff(PEEP_crossings_idx_sub) != 0))
    P_occ_starts[idx] = int(PEEP_crossings_idx[a])
    P_occ_ends[idx] = int(PEEP_crossings_idx[a+1])

In [ ]:
# Data sanity check: Make sure occlusion manoeuvres have been detected
if len(P_occ_starts) == 0 or len(P_occ_peaks) ==0  or len(P_occ_ends) == 0:
    print('No occlussion peaks were detected in the selected window,' 
          + 'please examine the data. Selected another window or file, '
          + 'accordingly')
else:
    print('You may continue!')

EMG peak identification

In [ ]:
# EMG peak detection parameters:

# Threshold peak height as fraction of max peak height 
# EMG_di_prominence_factor = 0.5 (default value)
EMG_di_prominence_factor = EMG_di_prominence_factor_default

# Threshold peak height as fraction of max peak height
# EMG_para_prominence_factor = 0.5 (default value)
EMG_para_prominence_factor = EMG_para_prominence_factor_default

# Draeger delay
# vent_delay = 0.5 (default value)
vent_delay = time_shift_default = 0.5                   

emg_peak_width = 0.2
percentile_border = 90

In [ ]:
# Find diaphragm EMG peaks and baseline crossings using the old baseline

y_di_RMS = RMS_data_emg_di[int(start_s):int(end_s)]
treshold = 0
width = int(emg_peak_width * emg_sample_rate)
prominence = EMG_di_prominence_factor * \
    (np.nanpercentile(y_di_RMS - emg_di_rolling_base_line_old, 75) 
     + np.nanpercentile(y_di_RMS - emg_di_rolling_base_line_old, 50))
EMG_peaks_di, properties = scipy.signal.find_peaks(
    y_di_RMS, height=treshold, prominence=prominence, width=width)

# Link EMG peak closest to occlusion pressures
EMG_di_occ_peaks = np.zeros((len(P_occ_peaks),), dtype=int)
PKS_idx = np.zeros((len(P_occ_peaks),), dtype=int)
for idx in range(len(P_occ_peaks)):
    P_occ_peak_converted = P_occ_peaks[idx]/vent_sample_rate*emg_sample_rate
    PKS_idx[idx] = np.argmin(
        np.abs(EMG_peaks_di + vent_delay * emg_sample_rate 
               - P_occ_peak_converted)
    )

    EMG_di_occ_peaks[idx] = EMG_peaks_di[PKS_idx[idx]]

# Old method
baseline_crossings_idx = np.argwhere(
    np.diff(np.sign(y_di_RMS - emg_di_rolling_base_line_old)) != 0)

EMG_di_occ_starts_old_base = np.zeros((len(P_occ_peaks),), dtype=int)
EMG_di_occ_ends_old_base = np.zeros((len(P_occ_peaks),), dtype=int)
for idx in range(len(P_occ_peaks)):
    baseline_crossings_idx_sub = np.sign(
        y_di_RMS[:EMG_di_occ_peaks[idx]] 
        - emg_di_rolling_base_line_old[:EMG_di_occ_peaks[idx]]
    )
    a = np.argmin(
        EMG_di_occ_peaks[idx] 
        - np.argwhere(np.diff(baseline_crossings_idx_sub) != 0))
    
    EMG_di_occ_starts_old_base[idx] = int(baseline_crossings_idx[a])
    
    if a < len(baseline_crossings_idx) - 1:
        EMG_di_occ_ends_old_base[idx] = int(baseline_crossings_idx[a+1])
    else:
        EMG_di_occ_ends_old_base[idx] = len(y_di_RMS) - 1 

In [ ]:
# Find diaphragm EMG peaks and baseline crossings using the new baseline

y_di_RMS = RMS_data_emg_di[int(start_s):int(end_s)]
treshold = 0
width = int(emg_peak_width * emg_sample_rate)
prominence = EMG_di_prominence_factor * \
    (np.nanpercentile(y_di_RMS - emg_di_rolling_base_line, 75) 
     + np.nanpercentile(y_di_RMS - emg_di_rolling_base_line, 50))
EMG_peaks_di, properties = scipy.signal.find_peaks(
    y_di_RMS, height=treshold, prominence=prominence, width=width)

# Link EMG peak closest to occlusion pressures
EMG_di_occ_peaks = np.zeros((len(P_occ_peaks),), dtype=int)
PKS_idx = np.zeros((len(P_occ_peaks),), dtype=int)
for idx in range(len(P_occ_peaks)):
    P_occ_peak_converted = P_occ_peaks[idx]/vent_sample_rate*emg_sample_rate
    PKS_idx[idx] = np.argmin(
        np.abs(EMG_peaks_di + vent_delay * emg_sample_rate 
               - P_occ_peak_converted)
    )

    EMG_di_occ_peaks[idx] = EMG_peaks_di[PKS_idx[idx]]

emg_di_rolling_base_line_aug = emg_di_rolling_base_line * (1+ 
    np.nanmedian(y_di_RMS[EMG_di_occ_peaks]) * di_baseline_std 
    / di_baseline_mean **2
)
    

baseline_crossings_idx = np.argwhere(
    np.diff(np.sign(y_di_RMS - emg_di_rolling_base_line_aug)) != 0)

EMG_di_occ_starts_new = np.zeros((len(P_occ_peaks),), dtype=int)
EMG_di_occ_ends_new = np.zeros((len(P_occ_peaks),), dtype=int)
for idx in range(len(P_occ_peaks)):
    baseline_crossings_idx_sub = np.sign(
        y_di_RMS[:EMG_di_occ_peaks[idx]] 
        - emg_di_rolling_base_line_aug[:EMG_di_occ_peaks[idx]]
    )
    a = np.argmin(
        EMG_di_occ_peaks[idx] 
        - np.argwhere(np.diff(baseline_crossings_idx_sub) != 0))
    
    EMG_di_occ_starts_new[idx] = int(baseline_crossings_idx[a])
    
    if a < len(baseline_crossings_idx) - 1:
        EMG_di_occ_ends_new[idx] = int(baseline_crossings_idx[a+1])
    else:
        EMG_di_occ_ends_new[idx] = len(y_di_RMS) - 1 

In [ ]:
# Plot new baseline crossing detection diaphragm

fig_5, axis = plt.subplots(nrows=2, ncols=max([len(
    P_occ_peaks),2]), figsize=(12, 6), sharex='col', sharey='row')

# Show EMG and Paw data for entire window
t_plot_start = np.min(
    [[P_occ_starts / vent_sample_rate], 
     [EMG_di_occ_starts_new / emg_sample_rate],
     [EMG_di_occ_starts_old_base / emg_sample_rate]], axis=0)[0] - 0.5
t_plot_end = np.max(
    [[P_occ_ends / vent_sample_rate], 
     [EMG_di_occ_ends_new / emg_sample_rate], 
     [EMG_di_occ_ends_old_base / emg_sample_rate]], axis=0)[0] + 0.5

for idx in range(len(EMG_di_occ_peaks)):
    start_i = EMG_di_occ_starts_new[idx]
    end_i = EMG_di_occ_ends_new[idx]
    start_plot_i = max([int(t_plot_start[idx] * emg_sample_rate), 0])
    end_plot_i = min([int(t_plot_end[idx] * emg_sample_rate), len(y_di_RMS)-1])
    
    baseline_start_i = max([0, EMG_di_occ_peaks[idx] - 5*emg_sample_rate])
    baseline_end_i = min(
        [len(y_di_RMS) - 1, EMG_di_occ_peaks[idx] + 5*emg_sample_rate])
    
    axis[0, idx].grid(True)
    axis[0, idx].plot(t_emg[start_plot_i:end_plot_i+1], 
                      y_di_RMS[start_plot_i:end_plot_i+1])
    axis[0, idx].plot(t_emg[start_plot_i:end_plot_i+1],
                      emg_di_rolling_base_line_aug[start_plot_i:end_plot_i+1])
    axis[0, idx].plot(t_emg[start_plot_i:end_plot_i+1],
                      emg_di_rolling_base_line_old[start_plot_i:end_plot_i+1],
                      'k')
    
    axis[0, idx].plot(t_emg[EMG_di_occ_starts_old_base[idx]], 
                      y_di_RMS[EMG_di_occ_starts_old_base[idx]], '*g')
    axis[0, idx].plot(t_emg[EMG_di_occ_ends_old_base[idx]], 
                      y_di_RMS[EMG_di_occ_ends_old_base[idx]], '*g')
    axis[0, idx].plot(t_emg[EMG_di_occ_peaks[idx]],
                      y_di_RMS[EMG_di_occ_peaks[idx]], '*r')
    axis[0, idx].plot(t_emg[start_i], y_di_RMS[start_i], '*r')
    axis[0, idx].plot(t_emg[end_i], y_di_RMS[end_i], '*r')
    axis[0, idx].set_xlabel('t (s)')
    
axis[0, 0].set_ylabel('sEMGdi (uV)')


PTP_occs = np.zeros((len(P_occ_peaks),))
Paw_max = np.zeros((len(P_occ_peaks),))
PTP_occ_baseline = np.zeros((len(P_occ_peaks),))
for idx in range(len(P_occ_peaks)):
    start_i = P_occ_starts[idx]
    end_i = P_occ_ends[idx]
    start_plot_i = max([int(t_plot_start[idx] * vent_sample_rate), 0])
    end_plot_i = min(
        [int(t_plot_end[idx] * vent_sample_rate), len(x)-1])


    baseline_start_i = max([0, P_occ_peaks[idx] - 5*vent_sample_rate])
    baseline_end_i = min([len(x) - 1, P_occ_peaks[idx] + 5*vent_sample_rate])
    Paw_max[idx] = max(P_rolling_base_line[baseline_start_i:baseline_end_i])

    axis[1, idx].grid(True)
    axis[1, idx].plot(t_vent[start_plot_i:end_plot_i+1], 
                      x[start_plot_i:end_plot_i+1])
    axis[1, idx].plot(t_vent[start_plot_i:end_plot_i+1],
                      P_rolling_base_line[start_plot_i:end_plot_i+1])    
    axis[1, idx].plot([t_vent[start_i], t_vent[end_i+1]], 
                      [Paw_max[idx], Paw_max[idx]], 'c')
    axis[1, idx].plot([t_vent[start_i], t_vent[start_i]], 
                      [x[start_i], Paw_max[idx]], 'c')
    axis[1, idx].plot([t_vent[end_i], t_vent[end_i]], 
                      [x[end_i], Paw_max[idx]], 'c')


    axis[1, idx].plot(t_vent[P_occ_peaks[idx]], x[P_occ_peaks[idx]], '*r')
    axis[1, idx].plot(t_vent[start_i], x[start_i], '*r')
    axis[1, idx].plot(t_vent[end_i], x[end_i], '*r')

axis[1, 0].set_ylabel('Pocc (cmH2O)')

In [ ]:
# Determine electrophysiological interpeak distance
t_delta_ecg_med = np.median(np.array(t_emg)[ECG_peaks[1:]] 
                            - np.array(t_emg)[ECG_peaks[:-1]])
t_delta_di_med = np.median(np.array(y_emg)[1, EMG_peaks_di[1:]] 
                           - np.array(y_emg)[1, EMG_peaks_di[:-1]])

In [ ]:
# Determine pneumatic interpeak distance -> Successive occluded breaths?
V_t = y_vent[2, int(start_vent_s):int(end_vent_s)]
treshold = 0.25 * np.percentile(V_t, 90)
prominence = 0.10 * np.percentile(V_t, 90)
width = 0.25 * vent_sample_rate
resp_efforts_volume_1, _ = scipy.signal.find_peaks(
    V_t, height=treshold, prominence=prominence, width=width)

treshold = 0.5 * np.percentile(V_t[resp_efforts_volume_1], 90)
prominence = 0.5 * np.percentile(V_t, 90)
width = 0.25 * vent_sample_rate
resp_efforts_volume, _ = scipy.signal.find_peaks(
    V_t, height=treshold, prominence=prominence, width=width)

# Plot entire selected time window with indicated peaks
# Paw
fig_1, axis = plt.subplots(nrows=3, ncols=1, figsize=(12, 6))
axis[0].grid(True)
axis[0].plot([y / vent_sample_rate for y in range(len(x))], x)
axis[0].plot([y / vent_sample_rate for y in range(len(x))],
             P_rolling_base_line)
axis[0].plot(P_occ_peaks/vent_sample_rate, x[P_occ_peaks], "x", color="r")
axis[0].plot(P_occ_starts / vent_sample_rate,
             P_rolling_base_line[P_occ_starts], '*r')
axis[0].plot(P_occ_ends / vent_sample_rate,
             P_rolling_base_line[P_occ_ends], '*r')
axis[0].set_ylabel('Paw (cmH2O)')

# V_vent
x_vent = y_vent[2, int(start_vent_s):int(end_vent_s)]
axis[1].grid(True)
axis[1].plot([y / vent_sample_rate for y in range(len(x))], x_vent)
axis[1].set_ylabel('V (mL)')

double_dips = np.zeros((len(P_occ_peaks),))
for idx in range(len(P_occ_peaks)):
    if idx > 0 :
        intermediate_breaths = np.equal(
            (P_occ_peaks[idx-1] < resp_efforts_volume  ), 
            (resp_efforts_volume < P_occ_peaks[idx]))
        
        if len(intermediate_breaths[intermediate_breaths == True]) > 0:
            double_dips[idx] = False
        else:
            double_dips[idx] = True

        axis[1].plot(
            resp_efforts_volume[intermediate_breaths]/vent_sample_rate, 
            x_vent[resp_efforts_volume[intermediate_breaths]], "x", color="g")

    else:
        double_dips[idx] = False

# sEMGdi:
axis[2].grid(True)
N_samp = len(RMS_data_emg_di[int(start_s):int(end_s)])
axis[2].plot([y / emg_sample_rate for y in range(N_samp)],
             RMS_data_emg_di[int(start_s):int(end_s)])
axis[2].plot([y / emg_sample_rate for y in range(N_samp)],
             emg_di_rolling_base_line_aug[:int(end_s)-int(start_s)])
axis[2].plot(EMG_di_occ_peaks/emg_sample_rate,
             y_di_RMS[EMG_di_occ_peaks], "x", color="r")
axis[2].plot(EMG_di_occ_starts_new/ emg_sample_rate,
             y_di_RMS[EMG_di_occ_starts_new], '*r')
axis[2].plot(EMG_di_occ_ends_new / emg_sample_rate,
             y_di_RMS[EMG_di_occ_ends_new], '*r')
axis[2].set_ylabel('RMS{sEMGdi} (uV)')
axis[2].set_xlabel('t (s)')
axis[2].set_ylim([0, 1.2*max(y_di_RMS[EMG_di_occ_peaks])])

In [ ]:
# Subplot per indicated peak
# TODO Shift sEMG by RMS window?

fig_2, axis = plt.subplots(nrows=2, ncols=max([len(
    P_occ_peaks),2]), figsize=(12, 6), sharex='col', sharey='row')

# Show EMG and Paw data for entire window
t_plot_start = np.min([[P_occ_starts / vent_sample_rate], 
                    [EMG_di_occ_starts_new / emg_sample_rate]], axis=0)[0] - 0.5
t_plot_end = np.max([[P_occ_ends / vent_sample_rate], 
                     [EMG_di_occ_ends_new / emg_sample_rate]], axis=0)[0] + 0.5

ETP_di_occs = np.zeros((len(EMG_di_occ_peaks),))
y_di_min = np.zeros((len(EMG_di_occ_peaks),))
ETP_di_baseline = np.zeros((len(EMG_di_occ_peaks),))
ETP_di_baseline_old = np.zeros((len(EMG_di_occ_peaks),))
for idx in range(len(EMG_di_occ_peaks)):
    start_i = EMG_di_occ_starts_new[idx]
    end_i = EMG_di_occ_ends_new[idx]
    start_plot_i = max([int(t_plot_start[idx] * emg_sample_rate), 0])
    end_plot_i = min([int(t_plot_end[idx] * emg_sample_rate), len(t_emg)])
    
    axis[0, idx].grid(True)
    axis[0, idx].plot(t_emg[start_plot_i:end_plot_i], 
                      ecg_rms[start_plot_i+205:end_plot_i+205]/
                      max(ecg_rms[start_plot_i+205:end_plot_i+205])*
                      max(y_di_RMS[start_plot_i:end_i]), 
                      'tab:gray', linewidth=0.5)
    
    axis[0, idx].plot(t_emg[start_plot_i:end_plot_i+1], 
                      y_di_RMS[start_plot_i:end_plot_i+1])
    axis[0, idx].plot(t_emg[start_plot_i:end_plot_i+1],
                      emg_di_rolling_base_line[start_plot_i:end_plot_i+1])

    baseline_start_i = max([0, EMG_di_occ_peaks[idx] - 5*emg_sample_rate])
    baseline_end_i = min(
        [len(y_di_RMS) - 1, EMG_di_occ_peaks[idx] + 5*emg_sample_rate])
    y_di_min[idx] = min(y_di_RMS[baseline_start_i:baseline_end_i])

    axis[0, idx].plot([t_emg[start_i], t_emg[end_i+1]], 
                      [y_di_min[idx], y_di_min[idx]], 'c')
    axis[0, idx].plot([t_emg[start_i], t_emg[start_i]], 
                      [y_di_RMS[start_i], y_di_min[idx]], 'c')
    axis[0, idx].plot([t_emg[end_i], t_emg[end_i]], 
                      [y_di_RMS[end_i], y_di_min[idx]], 'c')

    ETP_di_baseline[idx] = np.trapz(
        emg_di_rolling_base_line_aug[start_i:end_i+1] 
        - y_di_min[idx], dx=1/emg_sample_rate
    )

    ETP_di_baseline_old[idx] = np.trapz(
        emg_di_rolling_base_line[start_i:end_i+1] - y_di_min[idx],
        dx=1/emg_sample_rate
    )

    axis[0, idx].plot(t_emg[EMG_di_occ_peaks[idx]],
                      y_di_RMS[EMG_di_occ_peaks[idx]], '*r')
    axis[0, idx].plot(t_emg[start_i], y_di_RMS[start_i], '*r')
    axis[0, idx].plot(t_emg[end_i], y_di_RMS[end_i], '*r')


    
    axis[0, idx].set_ylim([0, 1.2*max(y_di_RMS[EMG_di_occ_peaks])])

    # EMG Time Product (ETP) diaphragm
    ETP_di_occs[idx] = np.trapz(
        y_di_RMS[start_i:end_i+1] 
        - emg_di_rolling_base_line_aug[start_i:end_i+1],
        dx=1/emg_sample_rate
    ) + ETP_di_baseline[idx]

    axis[0, idx].set_xlabel('t (s)')

axis[0, 0].set_ylabel('sEMGdi (uV)')

PTP_occs = np.zeros((len(P_occ_peaks),))
Paw_max = np.zeros((len(P_occ_peaks),))
PTP_occ_baseline = np.zeros((len(P_occ_peaks),))

for idx in range(len(P_occ_peaks)):
    start_i = P_occ_starts[idx]
    end_i = P_occ_ends[idx]
    start_plot_i = max([int(t_plot_start[idx] * vent_sample_rate), 0])
    end_plot_i = min(
        [int(t_plot_end[idx] * vent_sample_rate), len(t_vent)])

    axis[1, idx].grid(True)
    axis[1, idx].plot(t_vent[start_plot_i:end_plot_i+1], 
                      x[start_plot_i:end_plot_i+1])
    axis[1, idx].plot(t_vent[start_plot_i:end_plot_i+1],
                      P_rolling_base_line[start_plot_i:end_plot_i+1])    

    baseline_start_i = max([0, P_occ_peaks[idx] - 5*vent_sample_rate])
    baseline_end_i = min([len(x) - 1, P_occ_peaks[idx] + 5*vent_sample_rate])
    Paw_max[idx] = max(P_rolling_base_line[baseline_start_i:baseline_end_i])

    axis[1, idx].plot([t_vent[start_i], t_vent[end_i+1]], 
                      [Paw_max[idx], Paw_max[idx]], 'c')
    axis[1, idx].plot([t_vent[start_i], t_vent[start_i]], 
                      [x[start_i], Paw_max[idx]], 'c')
    axis[1, idx].plot([t_vent[end_i], t_vent[end_i]], 
                      [x[end_i], Paw_max[idx]], 'c')
    
    PTP_occ_baseline[idx] = np.trapz(
        Paw_max[idx] - P_rolling_base_line[start_i:end_i+1],
        dx=1/vent_sample_rate
    )

    axis[1, idx].plot(t_vent[P_occ_peaks[idx]], x[P_occ_peaks[idx]], '*r')
    axis[1, idx].plot(t_vent[start_i], x[start_i], '*r')
    axis[1, idx].plot(t_vent[end_i], x[end_i], '*r')

    # Pressure Time Product (PTP) occlusion
    PTP_occs[idx] = np.abs(np.trapz(
        x[start_i:end_i+1]-P_rolling_base_line[start_i:end_i+1],
        dx=1/vent_sample_rate
    )) + PTP_occ_baseline[idx]

axis[1, 0].set_ylabel('Pocc (cmH2O)')

In [ ]:
# Calculated neuromuscular efficiency: NMCdi = PTPocc / ETP_di
NMC_di = np.abs(PTP_occs)/ETP_di_occs

# Calculate signal-to-noise ratios. 
#   As the true signal and noise are not known,
#   the SNR is estimated as the maximum divided by the median of the moving 
#   baseline in a 1 s window around the peak.

SNR_di = feat.snr_pseudo(
    src_signal=RMS_data_emg_di[int(start_s):int(end_s)], 
    peaks=EMG_di_occ_peaks, 
    baseline=emg_di_rolling_base_line_aug[:int(end_s)-int(start_s)]
)

In [ ]:
# Evaluate Pocc quality according to Paw up- and downslope

fig_4, axis_tmp = plt.subplots(nrows=2, ncols=max([len(
    P_occ_peaks),2]), figsize=(12, 6), sharex='row', sharey='row')

t_vent = np.array(t_vent)

n_bins = 20
dP_up_10 = np.zeros((len(P_occ_peaks),))
dP_up_90 = np.zeros((len(P_occ_peaks),))
for idx in range(len(P_occ_peaks)):

    start_i = P_occ_starts[idx]
    end_i = P_occ_ends[idx]
    start_plot_i = max([int(t_plot_start[idx] * vent_sample_rate), 0])
    end_plot_i = min(
        [int(t_plot_end[idx] * vent_sample_rate), len(x)-1])
    
    # Paw-time plots
    axis_tmp[0, idx].grid(True)
    axis_tmp[0, idx].plot(
        t_vent[start_plot_i:end_plot_i+1] - t_vent[start_plot_i], 
        x[start_plot_i:end_plot_i+1])
    axis_tmp[0, idx].plot(
        t_vent[start_plot_i:end_plot_i+1]- t_vent[start_plot_i],
        P_rolling_base_line[start_plot_i:end_plot_i+1])    

    baseline_start_i = max([0, P_occ_peaks[idx] - 5*vent_sample_rate])
    baseline_end_i = min([len(x) - 1, P_occ_peaks[idx] + 5*vent_sample_rate])

    axis_tmp[0, idx].plot(
        [t_vent[start_i], t_vent[end_i+1]] - t_vent[start_plot_i], 
        [Paw_max[idx], Paw_max[idx]], 'c')
    axis_tmp[0, idx].plot(
        [t_vent[start_i], t_vent[start_i]] - t_vent[start_plot_i], 
        [x[start_i], Paw_max[idx]], 'c')
    axis_tmp[0, idx].plot(
        [t_vent[end_i], t_vent[end_i]] - t_vent[start_plot_i], 
        [x[end_i], Paw_max[idx]], 'c')

    axis_tmp[0, idx].plot(t_vent[P_occ_peaks[idx]] - t_vent[start_plot_i],
                    x[P_occ_peaks[idx]], '*r')
    axis_tmp[0, idx].plot(t_vent[start_i] - t_vent[start_plot_i],
                      x[start_i], '*r')
    axis_tmp[0, idx].plot(t_vent[end_i] - t_vent[start_plot_i], 
                      x[end_i], '*r')
    axis_tmp[0, idx].set_xlabel('t (s)')

    dP_up_10[idx] = np.percentile(
        x[P_occ_peaks[idx]+1:end_i]-x[P_occ_peaks[idx]:end_i-1], 10)
    dP_up_90[idx] = np.percentile(
        x[P_occ_peaks[idx]+1:end_i]-x[P_occ_peaks[idx]:end_i-1], 90)
    
    # Histogram of dPocc in down- (blue) and upslope (red)
    axis_tmp[1, idx].grid(True)
    axis_tmp[1, idx].hist(
        x[start_i+1:P_occ_peaks[idx]+1]-x[start_i:P_occ_peaks[idx]], 
        bins=n_bins, color='b')

    axis_tmp[1, idx].grid(True)
    axis_tmp[1, idx].hist(
        x[P_occ_peaks[idx]+1:end_i]-x[P_occ_peaks[idx]:end_i-1], 
        bins=n_bins, color='r')

    axis_tmp[1, idx].set_xlabel('dPocc (cmH2O)')

# Make x-axes symmetric in y-axis
low, high = axis_tmp[1, 0].get_xlim()
bound = max(abs(low), abs(high))
axis_tmp[1, 0].set_xlim(-bound, bound)
axis_tmp[0, 0].set_ylabel('Pocc (cmH2O)')
axis_tmp[1, 0].set_ylabel('Sample count (.)')


In [ ]:
# Evaluate sEMG peak quality according to waveform morphology

fig_3, axis = plt.subplots(nrows=2, ncols=max([len(
    P_occ_peaks), 2]), figsize=(12, 6), sharex='col', sharey='row')

# Show EMG and Paw data for entire window
t_plot_start = np.min([[P_occ_starts / vent_sample_rate], 
                    [EMG_di_occ_starts_new/ emg_sample_rate]], axis=0)[0] - 0.5
t_plot_end = np.max([[P_occ_ends / vent_sample_rate], 
                     [EMG_di_occ_ends_new / emg_sample_rate]], axis=0)[0] + 0.5

from scipy.optimize import curve_fit

# Define the bell-curve function
def func(x, a, b, c):
    return a * np.exp(-(x-b)**2 / c**2)

# Evaluate bell-curve error of sEMGdi peaks
ETP_di_bell_error = np.zeros((len(EMG_di_occ_peaks),))
for idx in range(len(EMG_di_occ_peaks)):
    start_i = EMG_di_occ_starts_new[idx]
    end_i = EMG_di_occ_ends_new[idx]
    start_plot_i = max([int(t_plot_start[idx] * emg_sample_rate), 0])
    end_plot_i = min([int(t_plot_end[idx] * emg_sample_rate), len(y_di_RMS)-1])
    
    
    
    baseline_start_i = max([0, EMG_di_occ_peaks[idx] - 5*emg_sample_rate])
    baseline_end_i = min(
        [len(y_di_RMS) - 1, EMG_di_occ_peaks[idx] + 5*emg_sample_rate])
    y_di_min[idx] = min(y_di_RMS[baseline_start_i:baseline_end_i])

    # Too little samples to fit parameters to --> Add additional datapoints
    if end_i - start_i < 3:
        plus_idx = 3 - (end_i - start_i)
    else:
        plus_idx = 0
    popt, pcov = curve_fit(func, 
                    t_emg[start_i:end_i+1], 
                    y_di_RMS[start_i:end_i+1]-y_di_min[idx],
                    bounds=([0., t_emg[EMG_di_occ_peaks[idx]]-0.5, 0.], 
                            [60., t_emg[EMG_di_occ_peaks[idx]]+0.5, 0.5])
                    )

    ETP_di_bell_error[idx] = np.trapz(
        np.sqrt((y_di_RMS[start_i:end_i+1] - 
        (func(t_emg[start_i:end_i+1], *popt)+y_di_min[idx])) **2),
        dx=1/emg_sample_rate
    )

    # Plot the fitted bell-curve relative to the sEMG peak
    axis[0, idx].grid(True)
    # ECG RMS for reference in gray
    axis[0, idx].plot(t_emg[start_plot_i:end_plot_i], 
                      ecg_rms[start_plot_i+205:end_plot_i+205]/
                      max(ecg_rms[start_plot_i+205:end_plot_i+205])*
                      max(y_di_RMS[start_plot_i:end_i]), 
                      'tab:gray', linewidth=0.5)
    # sEMGdi peak with detected on- and offset and peak in red  
    axis[0, idx].plot(t_emg[start_plot_i:end_plot_i+1], 
                      y_di_RMS[start_plot_i:end_plot_i+1])
    # sEMGdi moving baseline
    axis[0, idx].plot(t_emg[start_plot_i:end_plot_i+1],
                      emg_di_rolling_base_line_aug[start_plot_i:end_plot_i+1])
    # sEMGdi peak, on- and off-set
    axis[0, idx].plot(t_emg[EMG_di_occ_peaks[idx]],
                      y_di_RMS[EMG_di_occ_peaks[idx]], '*r')
    axis[0, idx].plot(t_emg[start_i], y_di_RMS[start_i], '*r')
    axis[0, idx].plot(t_emg[end_i], y_di_RMS[end_i], '*r')
    # Fitted bell-curve in green
    axis[0, idx].plot(t_emg[start_plot_i:end_plot_i+1], 
                      func(t_emg[start_plot_i:end_plot_i+1], *popt)
                      + y_di_min[idx], 'g')
    

    axis[0, idx].set_ylim([0, 1.2*max(y_di_RMS[EMG_di_occ_peaks])])
    axis[0, idx].set_xlabel('t (s)')

axis[0, 0].set_ylabel('sEMGdi (uV)')

# Evaluate bell-curve error of Pocc peaks
PTP_occ_bell_error = np.zeros((len(EMG_di_occ_peaks),))
for idx in range(len(P_occ_peaks)):
    start_i = P_occ_starts[idx]
    end_i = P_occ_ends[idx]
    start_plot_i = max([int(t_plot_start[idx] * vent_sample_rate), 0])
    end_plot_i = min(
        [int(t_plot_end[idx] * vent_sample_rate), len(x)-1])

    baseline_start_i = max([0, P_occ_peaks[idx] - 5*vent_sample_rate])
    baseline_end_i = min([len(x) - 1, P_occ_peaks[idx] + 5*vent_sample_rate])

    popt, pcov = curve_fit(func, 
                        t_vent[start_i:end_i+1], 
                        -(x[start_i:end_i+1]-Paw_max[idx]),
                        bounds=([0., t_vent[P_occ_peaks[idx]]-0.5, 0.], 
                                [100., t_vent[P_occ_peaks[idx]]+0.5, 0.5])
                        )
    
    PTP_occ_bell_error[idx] = np.trapz(
        np.sqrt((x[start_i:end_i+1] - 
        (-func(t_vent[start_i:end_i+1], *popt)+Paw_max[idx])) **2),
        dx=1/vent_sample_rate
    )
    # Plot the fitted bell-curve relative to the Pocc peak
    axis[1, idx].grid(True)
    # Pocc peak with detected on- and offset and peak in red 
    axis[1, idx].plot(t_vent[start_plot_i:end_plot_i+1], 
                      x[start_plot_i:end_plot_i+1])
    # Pocc moving baseline
    axis[1, idx].plot(t_vent[start_plot_i:end_plot_i+1],
                      P_rolling_base_line[start_plot_i:end_plot_i+1])
    # Pocc peak, on- and offset
    axis[1, idx].plot(t_vent[P_occ_peaks[idx]], x[P_occ_peaks[idx]], '*r')
    axis[1, idx].plot(t_vent[start_i], x[start_i], '*r')
    axis[1, idx].plot(t_vent[end_i], x[end_i], '*r')
    
    # Fitted bell-curve in green
    axis[1, idx].plot(t_vent[start_plot_i:end_plot_i+1], 
                    -func(t_vent[start_plot_i:end_plot_i+1], *popt)
                    + P_rolling_base_line[start_plot_i:end_plot_i+1], 'g')

axis[1, 0].set_ylabel('Paw (cmH2O)')

In [ ]:
# TODO Criteria met: Y/N table-like?

# Store values for later analysis

In [ ]:
fig_1.savefig(main_output_dir + '/' + patient + '/' + measurement_date
              + '/' + measurement_date + '_' + patient + '_' + PEEP_step_chosen 
              + '_timeplots_'+str(int(gate_width*1000))+'.png', 
              dpi=300)
fig_2.savefig(main_output_dir + '/' + patient + '/' + measurement_date  
              + '/' + measurement_date + '_' + patient + '_' + PEEP_step_chosen 
              + '_snapshots_'+str(int(gate_width*1000))+'.png', 
              dpi=300)
fig_3.savefig(main_output_dir + '/' + patient + '/' + measurement_date  
              + '/' + measurement_date + '_' + patient + '_' + PEEP_step_chosen 
              + '_bell_errors_'+str(int(gate_width*1000))+'.png', 
              dpi=300)

fig_4.savefig(main_output_dir + '/' + patient + '/' + measurement_date  
              + '/' + measurement_date + '_' + patient + '_' + PEEP_step_chosen 
              + '_Pocc_slopes_'+str(int(gate_width*1000))+'.png', 
              dpi=300)

In [ ]:
# Store the output parameters in a dataframe
for idx in range(len(PTP_occs)):
    data_now = [patient, PEEP_step_chosen, PEEP_set, PTP_occs[idx],
                ETP_di_occs[idx], NMC_di[idx],
                SNR_di[idx],
                PTP_occ_baseline[idx], ETP_di_baseline[idx], 
                ETP_di_baseline_old[idx],
                PTP_occ_bell_error[idx], ETP_di_bell_error[idx], 
                t_delta_di_med, t_delta_ecg_med,
                P_occ_peaks[idx]/vent_sample_rate, 
                P_occ_starts[idx]/vent_sample_rate, 
                P_occ_ends[idx]/vent_sample_rate,
                EMG_di_occ_peaks[idx]/emg_sample_rate, 
                EMG_di_occ_starts_new[idx]/emg_sample_rate, 
                EMG_di_occ_ends_new[idx]/emg_sample_rate,
                dP_up_10[idx], dP_up_90[idx], double_dips[idx],
                ]
    
    big_data_list.append(data_now)

    columns_now = ['patient', 'measurement', 'PEEP_set', 'PTP_occs', 'ETP_di_occs', 
               'NMC_di', 'SNR_di',
               'PTP_occ_baseline', 'ETP_di_baseline', 'ETP_di_baseline_old', 
               'PTP_occ_bell_SE', 'ETP_di_bell_SE',
               't_delta_di_med', 't_delta_ecg_med', 
               't_Pocc_peak', 't_Pocc_start', 't_Pocc_end',
               't_di_peak', 't_di_start', 't_di_end',
               'dP_up_10', 'dP_up_90', 'double_dip'
               ]


In [ ]:
df = pd.DataFrame(big_data_list, columns=columns_now)
df

In [ ]:
bp = df.plot.scatter('PEEP_set', 'NMC_di')
bp.set_ylabel('NMC (cmH2O/uV)')
bp.set_xlabel('PEEP (cmH2O)')
bp.set_title('')

In [ ]:
# Generate read-me file and save it
if gate_twice == True:
    N_gated = 2
else:
    N_gated = 1

context = (patient + ' / ' + measurement_date + ' / ' + PEEP_step_chosen + '\n'
        +'t_start: ' + str(start) + ' s' + '\n'
        +'t_end: ' + str(end) + ' s' + '\n'
        +'gate_width: ' + str(int(gate_width*1000)) + ' ms' + '\n'
        +'gate_threshold: ' + str(peak_fraction) + '' + '\n'
        +'gate_ECG_shift: ' + str(ECG_shift) + ' samples' + '\n'
        +'time_shift: ' + str(vent_delay) + ' s' + '\n'
        +'P_occ_prominence_factor: ' + str(P_occ_prominence_factor) 
        + '' + '\n'
        +'EMG_di_prominence_factor: ' + str(EMG_di_prominence_factor) 
        + '' + '\n'
        +'PEEP_set: ' + str(PEEP_set) + '' + '\n' \
        +'N_gated: ' + str(N_gated) + '' + '\n' \
        +'augmented_percentile: ' + str(augmented_perc) + ' %' + '\n'
        +'analysis_date: ' + str(datetime.now()) + ''
        )

with open(main_output_dir + '/' + patient + '/' + measurement_date + '/' 
          + measurement_date + '_' + patient + '_' + PEEP_step_chosen 
          + '_readme.txt', 'w') as f:
    f.write(context)

print(context)

In [ ]:
# Store PEEP-trial data if all PEEP-levels have been analysed
if len(set(df['PEEP_set'].values)) < 4:
    print('Warning: Not 4 PEEP settings evaluated yet!')
else:
    df.to_csv(main_output_dir + '/' + patient + '/' + measurement_date  
              + '/' + measurement_date + '_' + patient
              +'_NMC_output_gate_'+str(int(gate_width*1000)) + '.csv')

    bp.figure.savefig(main_output_dir + '/' + patient + '/' 
                      + measurement_date  + '/' + measurement_date 
                      + '_' + patient + '_gate_'+str(int(gate_width*1000)) 
                      + '_scatter.png', dpi=300)
    
    print('Notification: Data of 4 PEEP settings stored!')
    print(measurement_date)

In [ ]:
# Move on to the next measurement date if data saved

if not os.path.exists(main_output_dir + '/' + patient + '/' 
                      + measurement_date  + '/' + measurement_date 
                      + '_' + patient + '_NMC_output_gate_'
                      + str(int(gate_width*1000))+'.csv'):
    print('Warning: Data not stored! Do not progress to next date!')
elif len(set(df['PEEP_set'].values)) < 4:
    print('Warning: Data not stored! Do not progress to next date!')
else:
    if date_idx < len(measurement_dates):
        print('Notification: Data appropriately stored. '
              +'You may progress to next date!')
    else:
        print('Notification: Data appropriately stored. \n'+
              'This was the last measurement of this patient. '
              +'You may progress to next patient!')